# Whaca Demo Notebook
## Basic Usage

In [1]:
from scripts.whaca import whaca

In [2]:
w = whaca(width_thresh=100,time_thresh=0.2,NFFT = 512)

In [3]:
# w.open_wav("smallneg.wav")
# w.open_url("https://sweet.tunes.org/")
w.open_wav("smallpos.wav")

In [4]:
%%time
import numpy as np
spec,freq,time = w.gen_spectro(process=False)

Wall time: 83 ms


In [5]:
#spec = w.filter_sounds(spec, time[1] - time[0])

In [6]:
import matplotlib.pyplot as plt
import numpy as np

plt.imshow(spec,interpolation="nearest")
plt.show()

<Figure size 640x480 with 1 Axes>

## Demo w/ Widgets

In [7]:
from ipywidgets import IntSlider, FloatSlider ,Text, interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [8]:
import matplotlib.pyplot as plt
import numpy as np

def make_spectrogram(db_t, time_t, width_t, NF, file):
    # set parameters
    w = whaca()
    w.open_wav(file)
    w.set_threshold_params(db_thresh=db_t, time_thresh=time_t, width_thresh=width_t)
    w.set_spectro_params(NFFT = 2 ** NF)
    # process
    spec,freq,time = w.gen_spectro(process=True)
    # detect sounds    
    spec = w.filter_sounds(spec, time[1] - time[0])
    # return plot
    plt.imshow(spec,interpolation="nearest")
    plt.colorbar()

In [10]:
l = widgets.Layout(width="75%")
s = {"description_width" : "initial"}


# this block looks ugly so the output looks pretty
interact(make_spectrogram, 
                db_t=IntSlider(value=3, min=0, max=15, step=1, description="Call Intensity Threshold",layout=l,style=s), 
                time_t=FloatSlider(value=0.2,min=0,max=1.0,step=0.02,description="Call Duration Threshold",layout=l,style=s), 
                width_t=IntSlider(value=200,min=0,max=1000,step=50,description="Broadband Bandwidth Threshold",layout=l,style=s), 
                NF=fixed(9),
                file=Text(value="smallpos.wav", placeholder="Type here!", description="File to analyze:",layout=l,style=s))

interactive(children=(IntSlider(value=3, description='Call Intensity Threshold', layout=Layout(width='75%'), m…

<function __main__.make_spectrogram(db_t, time_t, width_t, NF, file)>